In [4]:
import polars as pl
import statistics

df = pl.read_csv("../../assets/games.csv")
season_2016 = df.filter((pl.col("season") == 2016) & (pl.col("game_type") == "REG"))

season_2016

game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
str,i64,str,i64,str,str,str,str,i64,str,i64,str,i64,i64,i64,i64,i64,str,str,str,i64,str,i64,i64,str,str,f64,str,str,f64,str,str,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str
"""2016_01_CAR_DEN""",2016,"""REG""",1,"""2016-09-08""","""Thursday""","""20:30""","""CAR""",20,"""DEN""",21,"""Home""",1,41,0,2016090800,56901,null,"""201609080den""","""4094""",400874484,null,7,7,"""-150""","""136""",-3.0,"""101""","""-111""",40.5,"""-110""","""-100""",0,"""outdoors""","""grass""",85,10,"""00-0027939""","""00-0032156""","""Cam Newton""","""Trevor Siemian""","""Ron Rivera""","""Gary Kubiak""","""Gene Steratore""","""DEN00""","""Sports Authority Field at Mile…"
"""2016_01_TB_ATL""",2016,"""REG""",1,"""2016-09-11""","""Sunday""","""13:00""","""TB""",31,"""ATL""",24,"""Home""",-7,55,0,2016091100,56902,null,"""201609110atl""","""4095""",400874522,null,7,7,"""120""","""-133""",2.5,"""-102""","""-108""",46.5,"""-105""","""-105""",1,"""dome""","""fieldturf""",null,null,"""00-0031503""","""00-0026143""","""Jameis Winston""","""Matt Ryan""","""Dirk Koetter""","""Dan Quinn""","""Jeff Triplette""","""ATL00""","""Georgia Dome"""
"""2016_01_BUF_BAL""",2016,"""REG""",1,"""2016-09-11""","""Sunday""","""13:00""","""BUF""",7,"""BAL""",13,"""Home""",6,20,0,2016091101,56903,null,"""201609110rav""","""4096""",400874510,null,7,7,"""151""","""-167""",3.0,"""105""","""-116""",44.5,"""-108""","""-102""",0,"""outdoors""","""grass""",84,8,"""00-0028118""","""00-0026158""","""Tyrod Taylor""","""Joe Flacco""","""Rex Ryan""","""John Harbaugh""","""Brad Allen""","""BAL00""","""M&T Bank Stadium"""
"""2016_01_CHI_HOU""",2016,"""REG""",1,"""2016-09-11""","""Sunday""","""13:00""","""CHI""",14,"""HOU""",23,"""Home""",9,37,0,2016091102,56904,null,"""201609110htx""","""4097""",400874511,null,7,7,"""210""","""-235""",5.5,"""-107""","""-103""",43.0,"""100""","""-110""",0,"""closed""","""fieldturf""",null,null,"""00-0024226""","""00-0029682""","""Jay Cutler""","""Brock Osweiler""","""John Fox""","""Bill O'Brien""","""Peter Morelli""","""HOU00""","""NRG Stadium"""
"""2016_01_GB_JAX""",2016,"""REG""",1,"""2016-09-11""","""Sunday""","""13:00""","""GB""",27,"""JAX""",23,"""Home""",-4,50,0,2016091103,56905,null,"""201609110jax""","""4098""",400874486,null,7,7,"""-179""","""161""",-3.5,"""-100""","""-110""",47.0,"""-110""","""-100""",0,"""outdoors""","""grass""",90,3,"""00-0023459""","""00-0031407""","""Aaron Rodgers""","""Blake Bortles""","""Mike McCarthy""","""Gus Bradley""","""Carl Cheffers""","""JAX00""","""EverBank Field"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2016_17_ARI_LA""",2016,"""REG""",17,"""2017-01-01""","""Sunday""","""16:25""","""ARI""",44,"""LA""",6,"""Home""",-38,50,0,2017010113,57154,null,"""201701010ram""","""4347""",400874692,null,8,8,"""-285""","""252""",-7.0,"""102""","""-113""",39.0,"""-110""","""-100""",1,"""outdoors""","""grass""",59,0,"""00-0021429""","""00-0033106""","""Carson Palmer""","""Jared Goff""","""Bruce Arians""","""Jeff Fisher""","""Craig Wrolstad""","""LAX99""","""Los Angeles Memorial Coliseum"""
"""2016_17_KC_SD""",2016,"""REG""",17,"""2017-01-01""","""Sunday""","""16:25""","""KC""",37,"""SD""",27,"""Home""",-10,64,0,2017010114,57155,null,"""201701010sdg""","""4348""",400874696,null,7,8,"""-235""","""210""",-5.5,"""-107""","""-103""",45.0,"""-108""","""-102""",1,"""outdoors""","""grass""",59,10,"""00-0023436""","""00-0022942""","""Alex Smith""","""Philip Rivers""","""Andy Reid""","""Mike McCoy""","""Walt Anderson""","""SDG00""","""Qualcomm Stadium"""
"""

In [5]:
def get_team_stats(season_df):
    results = {}

    for row in season_df.iter_rows(named=True):
        away_team = row["away_team"]
        home_team = row["home_team"]
        away_score = row["away_score"]
        home_score = row["home_score"]

        # Initialize team records
        for team in [away_team, home_team]:
            if team not in results:
                results[team] = {
                    "games": [],
                    "points_for": [],
                    "points_against": [],
                    "wins": 0,
                    "losses": 0,
                    "ties": 0,
                }

        # Record game results
        if away_score > home_score:
            results[away_team]["games"].append(1)  # Win
            results[home_team]["games"].append(0)  # Loss
            results[away_team]["wins"] += 1
            results[home_team]["losses"] += 1
        elif home_score > away_score:
            results[home_team]["games"].append(1)  # Win
            results[away_team]["games"].append(0)  # Loss
            results[home_team]["wins"] += 1
            results[away_team]["losses"] += 1
        else:
            results[away_team]["games"].append(0.5)  # Tie
            results[home_team]["games"].append(0.5)  # Tie
            results[away_team]["ties"] += 1
            results[home_team]["ties"] += 1

        # Record scoring data
        results[away_team]["points_for"].append(away_score)
        results[away_team]["points_against"].append(home_score)
        results[home_team]["points_for"].append(home_score)
        results[home_team]["points_against"].append(away_score)

    return results

team_stats = get_team_stats(season_2016)

In [7]:
team_data = []
for team, stats in team_stats.items():
    avg_points_for = statistics.mean(stats["points_for"])
    avg_points_against = statistics.mean(stats["points_against"])
    point_diff = round(avg_points_for - avg_points_against, 1)

    # Calculate streak
    games = stats["games"]
    most_recent = games[-1]
    streak_count = 1

    # Count consecutive games of same result
    for i in range(len(games) - 2, -1, -1):
        if games[i] == most_recent:
            streak_count += 1
        else:
            break

    if most_recent == 1:
        streak = streak_count
    elif most_recent == 0:
        streak = -streak_count
    else:  # tie
        streak = 0

    team_data.append(
        {
            "Team": team,
            "Record": f"{stats['wins']}-{stats['losses']}-{stats['ties']}",
            "Wins": stats["wins"],
            "Avg PF": avg_points_for,
            "Avg PA": avg_points_against,
            "Point Diff": point_diff,
            "Games": list(stats["games"][:16]),
            "Streak": streak,
        }
    )

season_df = pl.DataFrame(team_data, strict=False)

season_df

Team,Record,Wins,Avg PF,Avg PA,Point Diff,Games,Streak
str,str,i64,f64,f64,f64,list[f64],i64
"""CAR""","""6-10-0""",6,23.0625,25.125,-2.1,"[0.0, 1.0, … 0.0]",-2
"""DEN""","""9-7-0""",9,20.8125,18.5625,2.2,"[1.0, 1.0, … 1.0]",1
"""TB""","""9-7-0""",9,22.125,23.0625,-0.9,"[1.0, 0.0, … 1.0]",1
"""ATL""","""11-5-0""",11,33.75,25.375,8.4,"[0.0, 1.0, … 1.0]",4
"""BUF""","""7-9-0""",7,24.9375,23.625,1.3,"[0.0, 0.0, … 0.0]",-2
…,…,…,…,…,…,…,…
"""ARI""","""7-8-1""",7,26.125,22.625,3.5,"[0.0, 1.0, … 1.0]",2
"""PIT""","""11-5-0""",11,24.9375,20.4375,4.5,"[1.0, 1.0, … 1.0]",7
"""WAS""","""8-7-1""",8,24.75,23.9375,0.8,"[0.0, 0.0, … 0.0]",-1


In [ ]:
teams_info = pl.read_csv("../../assets/teams_colors_logos.csv")

# Merge with team information
nfl_df = season_df.join(
    teams_info.select(
        [
            "team_abbr",
            "team_division",
            "team_conf",
            "team_name",
            "team_logo_espn",
        ]
    ),
    left_on="Team",
    right_on="team_abbr",
    how="left",
)

nfl_df = nfl_df.sort(
    by=["team_division", "Wins", "Point Diff"],
    descending=[False, True, True],
)

nfl_df

Team,Record,Wins,Avg PF,Avg PA,Point Diff,Games,Streak,team_division,team_conf,team_name,team_logo_espn
str,str,i64,f64,f64,f64,list[f64],i64,str,str,str,str
"""NE""","""14-2-0""",14,27.5625,15.625,11.9,"[1.0, 1.0, … 1.0]",7,"""AFC East""","""AFC""","""New England Patriots""","""https://a.espncdn.com/i/teamlo…"
"""MIA""","""10-6-0""",10,22.6875,23.75,-1.1,"[0.0, 0.0, … 0.0]",-1,"""AFC East""","""AFC""","""Miami Dolphins""","""https://a.espncdn.com/i/teamlo…"
"""BUF""","""7-9-0""",7,24.9375,23.625,1.3,"[0.0, 0.0, … 0.0]",-2,"""AFC East""","""AFC""","""Buffalo Bills""","""https://a.espncdn.com/i/teamlo…"
"""NYJ""","""5-11-0""",5,17.1875,25.5625,-8.4,"[0.0, 1.0, … 1.0]",1,"""AFC East""","""AFC""","""New York Jets""","""https://a.espncdn.com/i/teamlo…"
"""PIT""","""11-5-0""",11,24.9375,20.4375,4.5,"[1.0, 1.0, … 1.0]",7,"""AFC North""","""AFC""","""Pittsburgh Steelers""","""https://a.espncdn.com/i/teamlo…"
…,…,…,…,…,…,…,…,…,…,…,…
"""CAR""","""6-10-0""",6,23.0625,25.125,-2.1,"[0.0, 1.0, … 0.0]",-2,"""NFC South""","""NFC""","""Carolina Panthers""","""https://a.espncdn.com/i/teamlo…"
"""SEA""","""10-5-1""",10,22.125,18.25,3.9,"[1.0, 0.0, … 1.0]",1,"""NFC West""","""NFC""","""Seattle Seahawks""","""https://a.espncdn.com/i/teamlo…"
"""ARI""","""7-8-1""",7,26.125,22.625,3.5,"[0.0, 1.0, … 1.0]",2,"""NFC West""","""NFC""","""Arizona Cardinals""","""https://a.espncdn.com/i/teamlo…"


In [13]:
nfl_df.write_json("nfl_2011_stats.json")